<a href="https://colab.research.google.com/github/harishk30/RingGalaxiesCNNAnalysis/blob/main/GAN/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory = '/content/drive/MyDrive/FinalRing', label_mode = None, image_size = (64, 64), batch_size = 32, shuffle = True
).map(lambda x: x/255.0)

Found 1437 files belonging to 1 classes.


In [ ]:
from tensorflow.keras import layers 

In [ ]:
discriminator = keras.Sequential(
    [
     keras.Input(shape = (64, 64, 3)),
     layers.Conv2D(64, kernel_size = 4, strides = 2, padding = "same"),
     layers.LeakyReLU(0.2),
     layers.Conv2D(128, kernel_size = 4, strides = 2, padding = "same"),
     layers.LeakyReLU(0.2),
     layers.Conv2D(128, kernel_size = 4, strides = 2, padding = "same"),
     layers.LeakyReLU(0.2),
     layers.Flatten(),
     layers.Dropout(0.2),
     layers.Dense(1, activation = "sigmoid"),
    ]
)

print(discriminator.summary())

latent_dim = 128
generator = keras.Sequential (
    [
     layers.Input(shape = (latent_dim,)),
     layers.Dense(8*8*128),
     layers.Reshape((8, 8, 128)),
     layers.Conv2DTranspose(128, kernel_size = 4, strides = 2, padding = "same"),
     layers.LeakyReLU(0.2),
     layers.Conv2DTranspose(256, kernel_size = 4, strides = 2, padding = "same"),
     layers.LeakyReLU(0.2),
     layers.Conv2DTranspose(512, kernel_size = 4, strides = 2, padding = "same"),
     layers.LeakyReLU(0.2),
     layers.Conv2D(3, kernel_size = 5, padding = "same", activation = "sigmoid"),
    ]
)

print(generator.summary())

opt_gen = tf.keras.optimizers.Adam(1e-4)
opt_disc = tf.keras.optimizers.Adam(1e-4)
loss_fn = keras.losses.BinaryCrossentropy()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 flatten (Flatten)           (None, 8192)              0

In [ ]:
from tqdm import tqdm

In [ ]:
for epoch in range(100):
  for idx, real in enumerate(tqdm(dataset)):
    batch_size = real.shape[0]
    random_latent_vectors = tf.random.normal(shape = (batch_size, latent_dim))
    
    fake = generator(random_latent_vectors)

    if idx % 5 == 0:
      img = keras.preprocessing.image.array_to_img(fake[0])
      img.save(f"/content/generated_images/generated_img{epoch}_{idx}_.png")

    with tf.GradientTape() as disc_tape:
      loss_disc_real = loss_fn(tf.ones((batch_size, 1)), discriminator(real))
      loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), discriminator(fake)) 
      loss_disc = (loss_disc_real + loss_disc_fake)/2
    
    grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
    opt_disc.apply_gradients(
        zip(grads, discriminator.trainable_weights)
    )

    with tf.GradientTape() as gen_tape:
      fake = generator(random_latent_vectors)
      output = discriminator(fake)
      loss_gen = loss_fn(tf.ones(batch_size, 1), output)

    grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
    opt_gen.apply_gradients(
        zip(grads, generator.trainable_weights)
    )

100%|██████████| 45/45 [00:05<00:00,  8.03it/s]
